# Exercise 7
## 1. Chimpanzee

In [379]:
import numpy as np

In [380]:
def forwardBackward(A, B, o, p0):
    """
    Implementation of the forward-backward algorithm
    
    
    Args:
        Matrix A: matrix describing the transition probabilities of the hidden random variables
        Matrix B: matrix describing the probabilities for the observations. The B_ij row represents 
                  the probability that the j-th observation is made when the system is in the i-th hidden state
        Array o:  of size (T,) where T is the number of observations. The t-th entry is the index of the 
                  t-th observation state.
        Array p0: of size (n_states) where n_states is the number of hidden states. 
                  Initial value for the probability distribution
    """
    #Forward part
    n_states, n_events = np.shape(B)
    T = np.shape(o)[0]
    
    #create observation matrices
    O = np.empty(shape=(n_events, n_states, n_states))
    for i in range(n_events):
        O[i,:,:] = np.diag(B[:,i].T)
        
    #do the forward pass
    c = np.ones(shape=(T+1,))
    f = np.ones(shape=(T+1, n_states))
    f[0] = p0
    for i in range(T):
        f[i+1] = f[i]@A@O[o[i]]   #forward step
        c[i+1] = np.sum(f[i+1])
        f[i+1] = f[i+1] / c[i+1]   #normalization
        
    #Backward part
    #do the backward pass
    b = np.ones(shape=(T+1,n_states))
    for i in range(T):
        b[-i-2] = A@O[o[-i-1]]@b[-i-1].transpose()    #backward step and normalization
        b[-i-2] = b[-i-2]/np.sum(b[-i-2])

    P = f*b 
    scale = np.sum(P,axis=1)
    scale = scale.reshape((len(scale),1))
    scale = np.repeat(scale, 2, axis=1)
    P = P / scale
    
    return P

In [381]:
#Wikipedia example
A = np.array([[0.7,0.3],[0.3,0.7]])
B = np.array([[0.9,0.1],[0.2,0.8]])
p0 = np.array([0.5,0.5])
o = np.array([0,0,1,0,0])

print("result wikipedia example")
P = forwardBackward(A,B,o,p0)
print(P)

#exercise sheet example
A = np.array([[0.5,0.5],[0.5,0.5]])
B = np.array([[0.5,0.4,0.1],[0.4,0.1,0.5]])
p0 = np.array([0.5,0.5])
o = np.array([0, 1, 1, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 1, 1, 2])

print("\n result chimpanzee example")
P = forwardBackward(A,B,o,p0)
print(P)

result wikipedia example
[[0.64693556 0.35306444]
 [0.86733889 0.13266111]
 [0.82041905 0.17958095]
 [0.30748358 0.69251642]
 [0.82041905 0.17958095]
 [0.86733889 0.13266111]]

 result chimpanzee example
[[0.5        0.5       ]
 [0.55555556 0.44444444]
 [0.8        0.2       ]
 [0.8        0.2       ]
 [0.55555556 0.44444444]
 [0.16666667 0.83333333]
 [0.55555556 0.44444444]
 [0.16666667 0.83333333]
 [0.16666667 0.83333333]
 [0.16666667 0.83333333]
 [0.55555556 0.44444444]
 [0.16666667 0.83333333]
 [0.16666667 0.83333333]
 [0.16666667 0.83333333]
 [0.16666667 0.83333333]
 [0.16666667 0.83333333]
 [0.55555556 0.44444444]
 [0.55555556 0.44444444]
 [0.8        0.2       ]
 [0.8        0.2       ]
 [0.16666667 0.83333333]]


In [382]:
def viterbi(A,B,o,p0):
    """
    Implementation of the viterbi algorithm
    
    
    Args:
        Matrix A: matrix describing the transition probabilities of the hidden random variables
        Matrix B: matrix describing the probabilities for the observations. The B_ij row represents 
                  the probability that the j-th observation is made when the system is in the i-th hidden state
        Array o:  of size (T,) where T is the number of observations. The t-th entry is the index of the 
                  t-th observation state.
        Array p0: of size (n_states) where n_states is the number of hidden states. 
                  Initial value for the probability distribution
    """    
    n_states, n_events = np.shape(B)
    T = np.shape(o)[0]
    
    #create observation matrices
    O = np.empty(shape=(n_events, n_states, n_states))
    for i in range(n_events):
        O[i,:,:] = np.diag(B[:,i].T)

    theta = np.zeros(shape=(T,n_states))
    psi = np.zeros(shape=(T,n_states))
    
    #initialization
    theta[0,:] = p0@O[o[0]]
    psi[0,:] = 0
    
    #recursion
    for i in range(1,T):
        theta_rep = np.reshape(theta[i-1], (len(theta[i-1]),1)).repeat(2,axis=1)
        theta[i] = B[:,o[i]].T * np.max(theta_rep*A, axis=0)
        
        psi[i] = np.argmax(theta_rep*A, axis=0)

    #termination
    q = np.empty(T, dtype=int)
    
    q[-1] = np.argmax(theta[-1])
    for i in range(1,T):
        q[-i-1] = psi[i,q[-i]]
    
    return q
    

In [383]:
A = np.array([[0.5,0.5],[0.5,0.5]])
B = np.array([[0.5,0.4,0.1],[0.4,0.1,0.5]])
p0 = np.array([0.5,0.5])
o = np.array([0, 1, 1, 0, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 1, 1, 2])

Q = viterbi(A,B,o,p0)
print(Q)

[0 0 0 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1]


In [393]:
print("globally most probable path:")
print(Q)

print("\npath of locally most probable sates:")
Q_ = np.argmax(P[1:],axis=1)
print(Q_)

print("\ncorrespondence between both paths:")
print(np.equal(Q_, Q).astype(int))

globally most probable path:
[0 0 0 0 1 1 1 1 1 0 1 1 1 0 1 0 0 0 0 1]

path of locally most probable sates:
[0 0 0 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 0 1]

correspondence between both paths:
[1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1]


We use the Viterbi algorithm to find the global MAP solution and the forward-backward algorithm to find the local MAP solutions. We see that both paths show a high level of correspondence, but are NOT the same.

## Pairwise Marginals